تمرین 1.4.2


In [1]:
import numpy as np 
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.covariance import EmpiricalCovariance
from scipy.spatial import distance
from scipy.stats import multivariate_normal
from sklearn.metrics import accuracy_score

In [2]:
def generate_gauss_classes(m,S,P,N):
    j=np.random.choice(len(m), 1, p=P)
    X_data =np.random.multivariate_normal(m[j[0],:], S ,1).T
    Y_data =j[0]
    for i in range (1,N):
        j=np.random.choice(len(m), 1, p=P)
        X_data =np.append(X_data, np.random.multivariate_normal(m[j[0],:], S ,1).T,axis = 1)
        Y_data =np.append(Y_data,j[0])
    return X_data,Y_data


def gaussian_mle(data):                                                                                                                                                                               
    mu = data.mean(axis=0)                                                                                                                                                                            
    var = (data-mu).T @ (data-mu) / data.shape[0] 

    return mu, var  
    
m=np.array([[0, 0, 0], [1 ,2, 2],[ 3, 3, 4]])
S=np.array([[0.8, 0.2, 0.1],[0.2, 0.8, 0.2],[0.1, 0.2, 0.8]])
P=np.array( [1/3, 1/3 , 1/3])
N=1000

# rows = len(mat)
[x_train,y_train]=generate_gauss_classes(m,S,P,N)
[x_test,y_test]=generate_gauss_classes(m,S,P,N)



Step 1. computing the ML estimates of the mean values and covariance matrix (common to all three
classes)

In [3]:
class1 =x_train[:, np.where(y_train == 0)[0]]
class2 =x_train[:,np.where(y_train == 1)[0]]
class3 =x_train[:, np.where(y_train == 2)[0]]


mu1,var1 = gaussian_mle(class1.T)
mu2,var2 = gaussian_mle(class2.T)
mu3,var3 = gaussian_mle(class3.T)

S_hat=np.array((1/3)*(var1+var2+var3))
m_hat=np.array([mu1, mu2, mu3])
print(class1.shape,class2.shape,class3.shape)
# class1.shape

(3, 326) (3, 361) (3, 313)


Step 2. Euclidean distance classifier

In [4]:
def euc_dist(a,b):
    return  np.sqrt(np.sum(np.square(a-b)))

def euclidean_classifier(means,data):
    y_hat =np.zeros(len(data[0]))
    for i in range(len(data[0])):
        arr = [euc_dist(data[:,i],means.T[:,j]) for j in range(len(means.T))]
        y_hat[i]=np.argmin(arr)
    return y_hat
z_euclidean=euclidean_classifier(m_hat,x_test)


step3:Mahalanobis distance classifier

In [5]:

def mahalanobis_classifier(means,cov,data):
    VI=np.linalg.inv(cov)
    y_hat =np.zeros(len(data[0]))
    for i in range(len(data[0])):
        arr = [distance.mahalanobis(data[:,i],means.T[:,j], VI) for j in range(len(means.T))]
        y_hat[i]=np.argmin(arr)
    return y_hat
z_mahalanobis=mahalanobis_classifier(m_hat,S_hat,x_test)


Step 4. 

In [6]:
def bayes_classifier(means,cov,P,data):
    y_hat =np.zeros(len(data[0]))
    for i in range (len(data[0])):
        arr=[P[j]*multivariate_normal.pdf( data[:,i] , means.T[:,j], cov ) for j in range(len(means.T))]
        y_hat[i]=np.argmax(arr)
    return y_hat
z_bayesian=bayes_classifier(m,S,P,x_test)


step5

In [7]:

err_euclidean = np.mean(y_test != z_euclidean)
err_mahalanobis =np.mean(y_test!=z_mahalanobis)
err_bayesian = np.mean(y_test!= z_bayesian)
print("euclidian erroe is:{}\n   mahalanobis error is:{}\n   bayesian error is:{}".format(err_euclidean, err_mahalanobis,err_bayesian))


euclidian erroe is:0.079
   mahalanobis error is:0.089
   bayesian error is:0.091
